In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [14]:
df = pd.read_csv('./input/copper_returns_5d_final.csv')
df.describe()

,LMCADS03,LMCADY,DXY,SPX,BCOM,MXWD,XAU,XAG,LMCADY_acu_5d,LMCADY_std_5d
count,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000
mean,-0.000185,-0.000178,0.000007,-0.000252,0.000072,-0.000205,-0.000274,-0.000106,-0.000867,0.014218
std,0.016059,0.016464,0.004793,0.011794,0.010250,0.009880,0.010722,0.019587,0.034782,0.008721
min,-0.112019,-0.110645,-0.024921,-0.103782,-0.054910,-0.085172,-0.097378,-0.123485,-0.185825,0.000805
25%,-0.008649,-0.008880,-0.002635,-0.005469,-0.005554,-0.004878,-0.005978,-0.009770,-0.021102,0.008616
50%,0.000000,0.000000,0.000000,-0.000390,0.000000,-0.000643,-0.000497,-0.000771,-0.001844,0.012148
75%,0.007562,0.007760,0.002658,0.003949,0.005375,0.003807,0.004908,0.008074,0.017065,0.017520
max,0.109603,0.109134,0.027541,0.136158,0.066117,0.105134,0.099792,0.226116,0.259832,0.091981


In [15]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df.drop(['Date', 'LMCADY_std_5d', 'LMCADY_acu_5d'], axis=1))
print(scaled_features.shape)
scaled_features

(5539, 8)


array([[0.51105054, 0.50820092, 0.46040927, ..., 0.45376117, 0.47765483,
        0.37989689],
       [0.50214433, 0.50261079, 0.51343716, ..., 0.46015831, 0.47895721,
        0.35204686],
       [0.54819756, 0.54585012, 0.39836873, ..., 0.46113695, 0.53109618,
        0.36137883],
       ...,
       [0.45696148, 0.45698224, 0.56849407, ..., 0.35097823, 0.49387949,
        0.34883688],
       [0.44663175, 0.44252277, 0.56989819, ..., 0.42771448, 0.41879554,
        0.31110092],
       [0.42031426, 0.41407919, 0.27142421, ..., 0.32700141, 0.52319187,
        0.33065703]])

In [16]:
def crearSecuencias(data, n_steps):
    X, y = [], []
    try:
        data = data.values  # Asegurarse de que 'data' es un array de NumPy
    except:
        pass
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :-2])  # las variables excepto los target
        y.append(data[i, -2:])            # los target
    return np.array(X), np.array(y)

In [17]:
n_steps = 25  # ventana modificable
X, y = crearSecuencias(scaled_features, n_steps)
(X.shape, y.shape)


((5514, 25, 6), (5514, 2))

In [18]:
# verificar que haya secuencia
print(X[0])
print(X[1])

[[0.51105054 0.50820092 0.46040927 0.44423603 0.50432324 0.45376117]
 [0.50214433 0.50261079 0.51343716 0.44530546 0.40553668 0.46015831]
 [0.54819756 0.54585012 0.39836873 0.43838686 0.51559425 0.46113695]
 [0.49410693 0.49374752 0.36140662 0.41908875 0.4375622  0.41213303]
 [0.52996151 0.5302504  0.55430507 0.39575223 0.48212846 0.41553961]
 [0.5619997  0.56257651 0.43096597 0.40912256 0.46098249 0.43504885]
 [0.49701022 0.49452026 0.447426   0.40636799 0.41316342 0.42126043]
 [0.41368416 0.40945351 0.4621292  0.45972953 0.44443493 0.48198419]
 [0.52550391 0.52944731 0.36991284 0.44453469 0.44646772 0.46557395]
 [0.36897736 0.36294137 0.50655397 0.44057081 0.38628919 0.45033045]
 [0.5033649  0.5028788  0.45836768 0.38636589 0.46335062 0.40053972]
 [0.46712256 0.46496218 0.44538254 0.43721685 0.42892662 0.46484468]
 [0.53753126 0.53518463 0.49544358 0.45992226 0.47799601 0.45961024]
 [0.47228924 0.47030131 0.5769906  0.39002791 0.41787082 0.39867988]
 [0.46046999 0.45865672 0.55432041

In [19]:
y

array([[0.47584125, 0.35551757],
       [0.48477789, 0.40260268],
       [0.47073196, 0.29257905],
       ...,
       [0.49387949, 0.34883688],
       [0.41879554, 0.31110092],
       [0.52319187, 0.33065703]])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)


In [21]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [22]:
model = Sequential([

    Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=64, kernel_size=2, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=32, kernel_size=2, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Flatten(),

    Dense(100, activation='relu'),
    Dropout(0.5),

    Dense(50, activation='relu'),
    Dropout(0.5),

    Dense(2)
    
])

initial_learning_rate = 0.0001
optimizer = Adam(learning_rate=initial_learning_rate)

model.compile(optimizer=optimizer, loss='mse', metrics=[rmse])


In [23]:
history = model.fit(X_train, y_train, epochs=50, verbose=1, validation_split=0.15,batch_size=64)


Epoch 1/50
63/63 [==============================] - 6s 12ms/step - loss: 2.3208 - rmse: 1.5086 - val_loss: 0.1923 - val_rmse: 0.4385
Epoch 2/50
63/63 [==============================] - 0s 6ms/step - loss: 1.5341 - rmse: 1.2275 - val_loss: 0.2559 - val_rmse: 0.5058
Epoch 3/50
63/63 [==============================] - 0s 6ms/step - loss: 1.1897 - rmse: 1.0823 - val_loss: 0.3072 - val_rmse: 0.5542
Epoch 4/50
63/63 [==============================] - 0s 6ms/step - loss: 0.9688 - rmse: 0.9784 - val_loss: 0.3143 - val_rmse: 0.5606
Epoch 5/50
63/63 [==============================] - 0s 8ms/step - loss: 0.7754 - rmse: 0.8756 - val_loss: 0.2588 - val_rmse: 0.5087
Epoch 6/50
63/63 [==============================] - 0s 8ms/step - loss: 0.6357 - rmse: 0.7941 - val_loss: 0.1934 - val_rmse: 0.4397
Epoch 7/50
63/63 [==============================] - 1s 8ms/step - loss: 0.5576 - rmse: 0.7421 - val_loss: 0.1254 - val_rmse: 0.3540
Epoch 8/50
63/63 [==============================] - 0s 7ms/step - loss: 0.4

In [24]:
loss = model.evaluate(X_test, y_test)
print(f'Loss on test data: {loss}')


26/26 [==============================] - 0s 3ms/step - loss: 0.0176 - rmse: 0.1321
Loss on test data: [0.017575090751051903, 0.132104754447937]
